### Import libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from newspaper import Article
import datetime
import datetime
import pandas as pd
import time

### Configure WebDriver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--incognito')
options.add_argument('--ignore-certificate-errors')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36')

chrome_driver_path = r"C:\Users\aefig\Downloads\chromedriver-win64\chromedriver.exe"

In [3]:
driver = webdriver.Chrome(service=Service(chrome_driver_path), options = options)

### Import S&P500 table

In [4]:
df_SP500 = pd.read_csv('S&P500_List.csv', encoding='utf8')

In [5]:
SP500_symbol = df_SP500['Symbol'].to_list()
SP500_name = df_SP500['Security'].to_list()

### Create dictionary to save the news

In [7]:
date = datetime.date.fromisoformat('2021-01-05')

while date < datetime.date.fromisoformat('2021-01-16'):
    i_date = str(date)
    e_date = str(date + datetime.timedelta(days=1))

    print(i_date)
    daily_register_news = {
                            'Symbol': [],
                            'Name': [],
                            'Source_1': [],
                            'Name_1': [],
                            'Text_1': [],
                            'Source_2': [],
                            'Name_2': [],
                            'Text_2': [],
                            'Source_3': [],
                            'Name_3': [],
                            'Text_3': [],
                            'Source_4': [],
                            'Name_4': [],
                            'Text_4': [],
                            'Source_5': [],
                            'Name_5': [],
                            'Text_5': [],
                        }

    for i in range(len(SP500_symbol):
        if i % 50 == 0:
            print(f"{int(i/5)}%")
    
        stock = SP500_symbol[i]
        name = SP500_name[i]

        daily_register_news['Symbol'].append(stock)
        daily_register_news['Name'].append(name)

        url = "https://news.google.com/search?q=" + name + "%20after%3A" + i_date + "%20before%3A" + e_date + "&hl=en-US&gl=US&ceid=US%3Aen"

        # time.sleep(0.01)
        driver.get(url)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        allData = soup.find_all("article",{"class":"IFHyqb DeXSAc"})

        for j in range(5):
            try:
                data = allData[j]
                source = data.find('a').get('href')
                source = f"https://news.google.com{source[1:]}"
                name = data.find('a', {'class':'JtKRv'}).text
                name = name.replace(",", "")
                daily_register_news[f'Source_{j+1}'].append(source)
                daily_register_news[f'Name_{j+1}'].append(name)
                article = Article(source)
                try:
                    article.download()
                    article.parse()
                    text = article.text
                    text = text.replace(",", "")
                    daily_register_news[f'Text_{j+1}'].append(text)
                except Exception as error:
                    text = F"ERROR: {error}"
                    daily_register_news[f'Text_{j+1}'].append(text)
            except:
                daily_register_news[f'Source_{j+1}'].append('')
                daily_register_news[f'Name_{j+1}'].append('')
                daily_register_news[f'Text_{j+1}'].append('')
                

    file = fr"C:\Users\aefig\OneDrive\Escritorio\Tec\07_6to Trimestre\02_ProyectoIntegrador\NewsRegister\{i_date}.csv"

    df_news = pd.DataFrame(daily_register_news)
    df_news.to_csv(file, index=False)
        
    date += datetime.timedelta(days=1)

2021-01-05
0%
2021-01-06
0%
2021-01-07
0%
2021-01-08
0%
2021-01-09
0%


In [9]:
driver.close()